# Construction d'un tableau récapitulatif

Le but de ce notebook est de construire un pandas de taille (sous forme de matrice (nombre d'articles, nombres de champs lexicaux * 6) qui pour tous les articles récapitule ses "performances" dans différentes catégories, et ce pour chaque champ lexical déterminé à l'avance. 

Commençons par définir ces champs lexicaux, qui correspondent à des facteurs de risques. Ceux-ci sont suggérés par le site Kaggle (d'où est extraite la base de données). 
Ces facteurs sont : 
- Hypertension
- Diabetes
- Male gender
- Heart Disease
- COPD and respiratory system diseases
- Smoking Status
- Age
- Cerebrovascular disease
- Cardio- and cerebrovascular disease
- Cancer
- Chronic kidney diseases
- Drinking
- Overweight or obese
- Chronic liver disease

Il faut maintenant créer un "champ lexical" relatif à chacun de ces thèmes. Il est important d'avoir suffisament de synonymes pour ne pas manquer d'oublier des termes dans les articles. Ces derniers étant prétraités en amont, inutiles de faire attention aux majuscules et aux tirets. 

In [3]:
hypertension = ['hypertension', 'high blood pressure', 'hypertensive', 'high pressure', 'raised blood pressure', 'htn', 'hbp', 'ht', ]
diabetes = ['diabetes', 'high blood sugar', 'insulin resistance', 'diabetic', 'diabetics', 'dm']
male_gender = ['male gender', 'male', 'gender', 'sex', 'masculine', 'female']
heart_disease = ['cardiopathy', 'heart disease', 'heart', 'chd', 'arrhythmia', 'tachycardia', 'bradycardia', 'fibrillation', 'cardiomyopathy', 'infarction']
COPD_respiratory_system = ['copd', 'emphysema', 'bronchitis', 'asthma', 'bronchiectasis', 'respiratory', 'trachea', 'lung', 'lungs', 'pulmonary']
smoking_status = ['smoking', 'smoke', 'smoker', 'tobacco']
age = ['age', 'old', 'young', 'senior', 'child', 'children']
cerebrovascular_disease = ['cerebrovascular', 'embolism', 'ischemic', 'stroke', 'aneurysm', 'tia', 'subarachnoid']
cancer = ['cancer', 'leukemia', 'cancerology']
kidney_disease = ['kidney', 'edema', 'anemia', 'gfr', 'dialysis']
drinking = ['drinking', 'alcohol', 'alcoholic']
overweight = ['overweight', 'obesity', 'obese', 'bmi']
liver_disease = ['liver', 'fascioliasis', 'cirrhosis', 'hepatitis', 'psc']

## 0. Prétraitement des données

Afin de travailler sur des textes le plus efficacement possible, on va procéder à un prétraitement qui consiste :
- à récupérer les données nécessaires dans le dataset
- à en extraire texte, titre, abstract, 

## 1. La "diffusivité" d'un thème dans un article

Le but de cette partie est de travailler sur un corpus restreint de textes comportant déjà un mot-clé (ou un groupe de mots clés) correspondant à un facteur de risque et éventuellement prétraités. On parcourt chaque texte (titre et abstract inclus) et pour chaque texte, on sauvegarde les positions des mots choisis. On calcule ensuite la moyenne et l'écart-type de la position du mot.



In [ ]:
import numpy as np
import math
texts
# L est la liste de textes = liste de strings
motcle = 'smoke'
for text in texts:
    words = text.split()
    occurences = [i for i in range(len(words)) if words[i] == motcle]
    mean_pos = np.mean(occurences)
    ecart_pos = math.sqrt(np.var(occurences))
    score = ecart_pos / len(words) 

On considère que lorsque l'écart type est plus grand que la moitié du nombre de mots du texte, cela veut dire que le mot-clé est cité dans plus de la moitié de l'article et donc que l'article parle principalement du mot-clé.
